# PySpark: Machine Learning

## 1. Preliminary

### Libraries

In [6]:
from pyspark.sql import SparkSession

### Spark Session

In [8]:
spark = SparkSession.builder.appName('training').getOrCreate()
spark

## 2. Logistic Regression

### Libraries

In [18]:
from pyspark.ml.classification import LogisticRegression

## 2. Load Dataset

In [11]:
df_spark = spark.read.csv('data/titanic.csv', header=True, inferSchema=True)

In [12]:
df_spark.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [13]:
df_spark.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

## 3. Data preparation

### String Indexer

In [21]:
from pyspark.ml.feature import StringIndexer

In [22]:
indexer = StringIndexer(inputCol='Sex', outputCol='Sex_indexed')

In [25]:
indexer.fit(df_spark).transform(df_spark).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Sex_indexed|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|        0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|        1.0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|        1.0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|        1.0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|      

### Vector Assembler

In [28]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [74]:
df_spark = spark.read.csv('data/iris.csv', header=True, inferSchema=True)
df_spark.columns
assembler = VectorAssembler(inputCols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol='features')
assembler.transform(df_spark).show()

+------------+-----------+------------+-----------+-------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|         features|
+------------+-----------+------------+-----------+-------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|[5.0,3.4,1.5,0.2]|
|         4.4|        2.9|         1.4|        0.2| Setosa|[4.4,2.9,1.4,0.2]|
|         4.9|        3.1|         1.5|        0.1| Setosa|[4.9,

### Scaler

In [30]:
from pyspark.ml.feature import StandardScaler

In [69]:
df_spark = spark.read.csv('data/iris.csv', header=True, inferSchema=True)
df_spark.columns

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety']

In [70]:
assembler = VectorAssembler(inputCols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol='features')
temp_df = assembler.transform(df_spark)

In [71]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')

In [72]:
scaler.fit(temp_df).transform(temp_df).select('features_scaled').show()

+--------------------+
|     features_scaled|
+--------------------+
|[6.15892840883878...|
|[5.9174018045706,...|
|[5.67587520030241...|
|[5.55511189816831...|
|[6.03816510670469...|
|[6.52121831524107...|
|[5.55511189816831...|
|[6.03816510670469...|
|[5.31358529390013...|
|[5.9174018045706,...|
|[6.52121831524107...|
|[5.79663850243650...|
|[5.79663850243650...|
|[5.19282199176603...|
|[7.00427152377744...|
|[6.88350822164335...|
|[6.52121831524107...|
|[6.15892840883878...|
|[6.88350822164335...|
|[6.15892840883878...|
+--------------------+
only showing top 20 rows

